In [7]:
from collections import defaultdict
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow
import tensorflow_hub as hub

from sklearn import preprocessing
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import f1_score
from sklearn.metrics import mean_absolute_error

import tensorflow_hub as hub
from verstack.stratified_continuous_split import scsplit


## Loading the Language model

Need some time

In [8]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [31]:
# test the language model

phrases_embed = embed(["The capital of France is Paris", "Paris is the biggest french city"])
print( "Similarity: ", ( sum( phrases_embed[0] * phrases_embed[1] ).numpy() ) )

phrases_embed = embed(["The capital of France is Paris", "I would like some coffee"])
print( "Similarity: ", sum( phrases_embed[0] * phrases_embed[1] ).numpy() )

phrases_embed = embed(["Some black tea with sugar, please", "I would like some coffee"])
print( "Similarity: ", sum( phrases_embed[0] * phrases_embed[1] ).numpy() )

Similarity:  0.7514804
Similarity:  0.06448189
Similarity:  0.4482424


## Loading the data

In [28]:
train_data = pd.read_csv("./data/train.csv")
X_train, X_test, y_train, y_test = scsplit(train_data, train_data['retweet_count'], stratify=train_data['retweet_count'], train_size=0.75, test_size=0.25)
X_train = X_train.drop(['retweet_count'], axis=1)
X_test = X_test.drop(['retweet_count'], axis=1)

X_train = X_train["text"]
X_test = X_test["text"]
print(X_train[:10])
print(embed(X_train[:10]))

210088                                           CALLED IT.
353448    first, manufacture the evidence, sell it to al...
110884    The best part of Trump’s Fox News town hall is...
118100    Because it’s Africa...the world won’t take it ...
213235    Troubling development out of Trousdale Correct...
41954                                             Ya think.
493307               This is so sad! He was the best of us.
99564                                     2020 is fucked up
248579     The rich are getting richer off the Corona Virus
377307    How long until we admit coronavirus has direct...
Name: text, dtype: object
tf.Tensor(
[[ 0.00281394  0.00676178  0.00043603 ... -0.01381203  0.01512007
   0.00630505]
 [-0.01473356 -0.06944113  0.0046551  ... -0.00106915 -0.0253217
  -0.01710856]
 [ 0.00447236 -0.00602912 -0.03080316 ... -0.00867151 -0.0626659
   0.05444583]
 ...
 [-0.06370888 -0.06671701  0.00352778 ... -0.02371787  0.00076623
  -0.02998426]
 [ 0.01011501 -0.04472246  0.0177922

## Embed the Texts

In [55]:
X_train = np.array(X_train)

In [58]:
X_train

array(['CALLED IT.',
       'first, manufacture the evidence, sell it to all the frauds, compare notes with each other, and call it verified.',
       'The best part of Trump’s Fox News town hall is the long commercial breaks.',
       ...,
       'On the front line - Taoiseach @LeoVaradkar at Morgans Place site in Blachardstown where he he help with #Covid19 testing of residents. This sends out a really positive message 👏🏻👏🏻👏🏻 https://t.co/neqoZ817dZ',
       'பெருநகர சென்னை மாநகராட்சியில், கொரோனா பாதிப்பில் இருந்து குணமடைந்தவர்கள், தற்போது சிகிச்சை பெருபவர்களின் மண்டலவாரி விவரம் #polimer #polimernews #coronavirus #Covid19 #GreaterChennaiCorporation https://t.co/ZWeX88fALk',
       '#Ofcom are a joke not fit for purpose what did they do about #FakeNews at the #BBC to justify bombing #Syria and #Iraq? If #5G turns out to be unsafe and people are harmed this woman and Ofcom will be finished. #NHS #BarCouncil #MisconductinPublicOffice #coronavirus #Rothschild https://t.co/c2mMOWFSxN'],
 

In [62]:
X_train.shape[0]

499332

In [64]:
X_train_embed = np.zeros( (X_train.shape[0], 512) )

In [71]:
for index, x in enumerate(X_train):
    if index % 1000 == 0:
        print(index)
    X_train_embed[index, :] = embed([x]).numpy()[0]

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


KeyboardInterrupt: 

In [73]:
X_train_embed = X_train_embed[:10000, :]

In [75]:
X_train_embed.shape

(10000, 512)

In [79]:
y_train = y_train[:10000]

In [80]:
y_train

array([ 0,  0, 89, ...,  0,  0,  1], dtype=int64)

## Build up the Neural Network

In [115]:
def build_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.Dense(1, 'relu'))
    model.compile(loss="mse", optimizer='adam', metrics=["mse"])
#     model.summary()
    return model

In [116]:
np.array( X_train_embed )

array([[ 0.00281393,  0.0067618 ,  0.00043604, ..., -0.01381206,
         0.01512006,  0.00630507],
       [-0.01473355, -0.06944113,  0.00465509, ..., -0.00106913,
        -0.02532168, -0.01710855],
       [ 0.00447234, -0.00602915, -0.03080317, ..., -0.00867152,
        -0.06266589,  0.05444582],
       ...,
       [-0.04201492,  0.03147441,  0.03473513, ...,  0.06916818,
         0.00319515, -0.1078413 ],
       [-0.07394182, -0.00274012,  0.08254025, ..., -0.05593901,
        -0.02026385, -0.02183842],
       [ 0.01431507,  0.06577744, -0.00417293, ..., -0.02583069,
        -0.07258449, -0.04139128]])

In [124]:
epochs = 50
batch_size = 256

In [125]:
model = build_model()
model.fit(X_train_embed, y_train, epochs=epochs, batch_size = batch_size, validation_split=0.2, shuffle=True)

Train on 8000 samples, validate on 2000 samples
Epoch 1/50
8000/8000 [==============================] - 1s 158us/sample - loss: 7768035.9216 - mse: 7768036.0000 - val_loss: 1476889.8475 - val_mse: 1476889.8750
Epoch 2/50
8000/8000 [==============================] - 0s 32us/sample - loss: 7709581.3765 - mse: 7709582.0000 - val_loss: 1468968.9283 - val_mse: 1468968.8750
Epoch 3/50
8000/8000 [==============================] - 0s 32us/sample - loss: 7596514.0077 - mse: 7596514.0000 - val_loss: 1557479.4981 - val_mse: 1557479.3750
Epoch 4/50
8000/8000 [==============================] - 0s 32us/sample - loss: 6716366.0275 - mse: 6716366.0000 - val_loss: 2400466.5145 - val_mse: 2400466.7500
Epoch 5/50
8000/8000 [==============================] - 0s 31us/sample - loss: 2421334.1502 - mse: 2421334.0000 - val_loss: 2017700.7858 - val_mse: 2017700.7500
Epoch 6/50
8000/8000 [==============================] - 0s 32us/sample - loss: 1953517.2414 - mse: 1953517.2500 - val_loss: 2235880.2734 - val_mse

In [126]:
model.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_74 (Dense)             multiple                  131328    
_________________________________________________________________
dense_75 (Dense)             multiple                  65792     
_________________________________________________________________
dense_76 (Dense)             multiple                  65792     
_________________________________________________________________
dense_77 (Dense)             multiple                  65792     
_________________________________________________________________
dense_78 (Dense)             multiple                  65792     
_________________________________________________________________
dense_79 (Dense)             multiple                  65792     
_________________________________________________________________
dense_80 (Dense)             multiple                

In [130]:
model.predict([[X_train_embed[2]]])

array([[15.925902]], dtype=float32)

In [131]:
y_train[2]

89